<a href="https://colab.research.google.com/github/fvillarino/Kaggle-Rossmann/blob/master/Rossmann_Fullmodels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
import pandas as pd
import numpy as np
import datetime

#### Datos normalizados

In [ ]:
from google.colab import drive

folder='drive/MyDrive/Colab Notebooks/Kaggle/Rossman/'
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_feather(folder+'train_normalized_data.fth')
df_test = pd.read_feather(folder+'test_normalized_data.fth')

#### Separo categoricas y continuas

In [ ]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

In [ ]:
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 'Precipitationmm',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'Promo', 'SchoolHoliday', 'StateHoliday_bool']

In [ ]:
y_out_columns = ['Sales']

### LightGBM

#### Split de datos para entrenar

In [ ]:
# split_type = 'random'
# split_type = 'no_split'
split_type = 'last_week'

In [ ]:
if split_type == 'no_split':
    df_train = df
elif split_type == 'last_week':
    # Esto divide en train y val
    df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
    df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
    print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_val)/(len(df_train) + len(df_val))}')
elif split_type == 'random':
    # Splitting aleatorio
    np.random.seed(42)
    indexes = np.arange(len(df))
    np.random.shuffle(indexes)
    N = len(df)//5
    df_train = df[N:]
    df_val = df[:N]
    print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_val)/(len(df_train) + len(df_val))}')

Cantidad en val: 30188, porcentaje: 0.035753454185409164


In [ ]:
X_train = df_train[cat_vars + contin_vars]
if split_type != 'no_split':
    X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

#### Normalización de los datos

In [ ]:
log_output = False
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    if split_type != 'no_split':
        y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    y_mean = df_train[y_out_columns].mean().values
    y_std = df_train[y_out_columns].std().values
    y_train = (df_train[y_out_columns].values - y_mean)/y_std
    if split_type != 'no_split':
        y_val = (df_val[y_out_columns].values - y_mean)/y_std

#### Armado del modelo

In [ ]:
from sklearn.model_selection import cross_val_score
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from lightgbm import LGBMRegressor

In [ ]:
def calculate_RMSE(X, y, log_output=True):
    y_preds = np.exp(model.predict(X, verbose=1)*max_log_y)
    return np.sqrt((((y - y_preds)/y)**2).sum()/len(y_preds))

In [ ]:
min_child_samples=5
n_estimators=2000
learning_rate=0.25
model = LGBMRegressor(min_child_samples=min_child_samples, n_estimators=n_estimators, learning_rate=learning_rate )

In [ ]:
fit_params={"early_stopping_rounds":100, 
            "eval_metric" : 'l2', 
            "eval_set" : [(X_val, y_val.reshape(-1))],
            'eval_names': ['valid'],
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': cat_vars
           }

In [ ]:
model.fit(X_train, y_train.reshape(-1), **fit_params)

/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Assortment', 'CompetitionMonthsOpen', 'CompetitionOpenSinceYear', 'Day', 'DayOfWeek', 'Events', 'Month', 'Promo2SinceYear', 'Promo2Weeks', 'PromoInterval', 'Promo_bw', 'Promo_fw', 'SchoolHoliday_bw', 'SchoolHoliday_fw', 'State', 'StateHoliday', 'StateHoliday_bool_bw', 'StateHoliday_bool_fw', 'Store', 'StoreType', 'Week', 'Year']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 100 rounds.
[100]	valid's l2: 0.100789	valid's l2: 0.100789
[200]	valid's l2: 0.0959811	valid's l2: 0.0959811
[300]	valid's l2: 0.097763	valid's l2: 0.097763
Early stopping, best iteration is:
[212]	valid's l2: 0.0943812	valid's l2: 0.0943812


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.25, max_depth=-1,
              min_child_samples=5, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=2000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

#### Análisis de la métrica

In [ ]:
model.score(X_val, y_val)

0.894198929653378

In [ ]:
if log_output:
    y_pred_train = np.exp(model.predict(X_train, verbose=1)*max_log_y)
    y_pred = np.exp(model.predict(X_val, verbose=1)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test, verbose=1)*max_log_y)
else:
    y_pred_train = model.predict(X_train, verbose=1)*y_std + y_mean
    y_pred = model.predict(X_val, verbose=1)*y_std + y_mean
    y_pred_test = model.predict(X_test, verbose=1)*y_std + y_mean

In [ ]:
# Train
np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train))

0.17867032619235249

In [ ]:
# Validación
np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))

0.14613309144773187

#### LightGBM con Hyperopt

In [ ]:
def predicctionDesNormalizacion(model, soloVal=False):
  if log_output:
      if soloVal:
        y_pred = np.exp(model.predict(X_val, verbose=1)*max_log_y)
      else:
        y_pred_train = np.exp(model.predict(X_train, verbose=1)*max_log_y)
        y_pred = np.exp(model.predict(X_val, verbose=1)*max_log_y)
        y_pred_test = np.exp(model.predict(X_test, verbose=1)*max_log_y)
  else:
      if soloVal:
        y_pred = model.predict(X_val, verbose=1)*y_std + y_mean
      else:
        y_pred_train = model.predict(X_train, verbose=1)*y_std + y_mean
        y_pred = model.predict(X_val, verbose=1)*y_std + y_mean
        y_pred_test = model.predict(X_test, verbose=1)*y_std + y_mean
  if soloVal:
    return y_pred
  return y_pred_train, y_pred, y_pred_test

In [ ]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from hyperopt import Trials
import hyperopt

In [ ]:
search_HyperOptSearch = {
    'learning_rate': hp.loguniform(low=0.01, high=1, label='learning_rate'),
    'max_depth':  hp.choice('max_depth', list(np.arange(5, 20))),
    #'num_leaves': hp.choice('num_leaves', list(np.arange(2, 150))), #max number of leaves in one tree        
    #'min_child_samples': hp.choice('min_child_samples', list(np.arange(0, 200))), # minimal number of data in one leaf
    #'reg_lambda': hp.loguniform('reg_lambda',1e-9, 1000), # L2 regularization
    #'reg_alpha': hp.loguniform('reg_alpha',1e-9, 1.0), # L1 regularization
    #'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0), # enabler of bagging fraction
    #'min_child_weight': hp.choice('min_child_weight', list(np.arange(0, 10,))), # minimal number of data in one leaf.
    'n_estimators': hp.choice('n_estimators', list(np.arange(900, 1000))) # cant. de estimadores secuenciales (se pone alto stopea earlystopping)    
}
max_iterSearch = 100

In [ ]:
def objective(params):
        
    clf = LGBMRegressor(**params)    
    score = -cross_val_score(clf, X_train.values, y_train.reshape(-1), cv=4, scoring='neg_root_mean_squared_error', verbose=1).mean()
    return score

tpe_trials = Trials()    


%time hyperOptLightGBM = fmin(fn=objective, space=search_HyperOptSearch, verbose=2, algo=hyperopt.tpe.suggest, max_evals=max_iterSearch)

  0%|          | 0/100 [00:00<?, ?it/s, best loss: ?]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



  1%|          | 1/100 [04:19<7:07:47, 259.27s/it, best loss: 0.5641907040185502]

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  4.3min finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



  2%|▏         | 2/100 [09:33<7:30:10, 275.62s/it, best loss: 0.5641907040185502]

...

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  4.7min finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



 33%|███▎      | 33/100 [2:21:56<5:06:30, 274.49s/it, best loss: 0.4233910643939531]

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  4.0min finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



 34%|███▍      | 34/100 [2:26:48<5:07:53, 279.91s/it, best loss: 0.4233910643939531]

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  4.9min finished

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



In [ ]:
print('best parameters:', hyperOptLightGBM)

In [ ]:
model = LGBMRegressor(**hyperOptLightGBM)
model.fit(X_train, y_train.reshape(-1), **fit_params)
y_pred_train, y_pred, y_pred_test = predicctionDesNormalizacion(model)

print('RMSE en train:',np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train)))
print('RMSE en val:', np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred)))